<a href="https://colab.research.google.com/github/nvarasp/Project_IRWA/blob/main/notebook/RQ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RQ2 - Output diversification

### 0. Imports and configuration

In [4]:
%%time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
CPU times: user 609 ms, sys: 144 ms, total: 754 ms
Wall time: 21.1 s


In [5]:
from google.colab import files
uploaded = files.upload()
for file in uploaded:
  print("file name : {} , length: {}".format(file,len(file)))

Saving functions_v3.py to functions_v3.py
file name : functions_v3.py , length: 15


In [6]:
%%capture
!pip install config

In [8]:
from functions_v3 import *
from gensim.models import Word2Vec
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from sklearn.cluster import KMeans
import seaborn as sns
from wordcloud import WordCloud, ImageColorGenerator
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import operator
import ast
STOPWORDS = set(stopwords.words("english"))
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
IMPORTS ="/content/drive/My Drive/IRWA/FinalProject/notebook/"  
CLUSTERS = '/content/drive/My Drive/IRWA/FinalProject/notebook/clustered_data.csv'

Read the processed data and generate index from it. Retrieve the term frequency, document frequency, and inverse document frequency.

In [9]:
df_proc = pd.read_csv(IMPORTS+'df_to_cluster.csv')
df_proc['Tweet_proc'] = [list(ast.literal_eval(x)) for x in df_proc['Tweet_proc']]
df_proc.head()

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,Tweet_proc
0,@JoeBiden R/T IF YOU AGREE!!\n#JoeBidenKamalaH...,SarxClothing_,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'JoeBidenKamalaHarris2020', 'indices...",0,0,https://twitter.com/twitter/statuses/133592651...,"[joebiden, rt, agree, joebidenkamalaharris2020..."
1,RT @dblackanese: It’s a very big deal. \n#Wear...,nightnum9,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'WearAMask', 'indices': [40, 50]}]",0,4400,https://twitter.com/twitter/statuses/133592651...,"[dblackanese, big, deal, wearamask, httpstcor9..."
2,RT @Amber02150: Wishing everyone a nice health...,jackwen5566,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'StaySafe', 'indices': [55, 64]}, {'...",0,400,https://twitter.com/twitter/statuses/133592651...,"[amber02150, wishing, everyone, nice, healthy,..."
3,RT @dblackanese: It’s a very big deal. \n#Wear...,jmcmichael2017,Mon Dec 07 12:37:56 +0000 2020,"[{'text': 'WearAMask', 'indices': [40, 50]}]",0,4400,https://twitter.com/twitter/statuses/133592651...,"[dblackanese, big, deal, wearamask, httpstcor9..."
4,@JoeBiden R/T IF YOU AGREE!!\n#JoeBidenKamalaH...,SarxClothing_,Mon Dec 07 12:37:57 +0000 2020,"[{'text': 'JoeBidenKamalaHarris2020', 'indices...",0,0,https://twitter.com/twitter/statuses/133592652...,"[joebiden, rt, agree, joebidenkamalaharris2020..."


In [10]:
start_time = time.time()
numDocuments = len(df_proc["Tweet_proc"])
index, tf, df, idf= create_tf_idf_index(df_proc,"Tweet_proc", numDocuments)

In [11]:
df_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16644 entries, 0 to 16643
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tweet       16644 non-null  object
 1   Username    16644 non-null  object
 2   Date        16644 non-null  object
 3   Hashtags    16644 non-null  object
 4   Likes       16644 non-null  int64 
 5   Retweets    16644 non-null  int64 
 6   Url         16644 non-null  object
 7   Tweet_proc  16644 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.0+ MB


### 1. Clustered data

In the previous research question, we decided to assign 4 clusters to our data, using the elbow criterion.

Read the processed data with the cluster label assigned.

In [108]:
df_clusters = pd.read_csv(CLUSTERS)
num_clusters = len(df_clusters.Cluster.unique())
df_clusters.head()

,Unnamed: 0,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,Tweet_proc,Cluster
0,0,@JoeBiden R/T IF YOU AGREE!!\n#JoeBidenKamalaH...,SarxClothing_,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'JoeBidenKamalaHarris2020', 'indices...",0,0,https://twitter.com/twitter/statuses/133592651...,"['joebiden', 'rt', 'agree', 'joebidenkamalahar...",1
1,1,RT @dblackanese: It’s a very big deal. \n#Wear...,nightnum9,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'WearAMask', 'indices': [40, 50]}]",0,4400,https://twitter.com/twitter/statuses/133592651...,"['dblackanese', 'big', 'deal', 'wearamask', 'h...",0
2,2,RT @Amber02150: Wishing everyone a nice health...,jackwen5566,Mon Dec 07 12:37:55 +0000 2020,"[{'text': 'StaySafe', 'indices': [55, 64]}, {'...",0,400,https://twitter.com/twitter/statuses/133592651...,"['amber02150', 'wishing', 'everyone', 'nice', ...",0
3,3,RT @dblackanese: It’s a very big deal. \n#Wear...,jmcmichael2017,Mon Dec 07 12:37:56 +0000 2020,"[{'text': 'WearAMask', 'indices': [40, 50]}]",0,4400,https://twitter.com/twitter/statuses/133592651...,"['dblackanese', 'big', 'deal', 'wearamask', 'h...",0
4,4,@JoeBiden R/T IF YOU AGREE!!\n#JoeBidenKamalaH...,SarxClothing_,Mon Dec 07 12:37:57 +0000 2020,"[{'text': 'JoeBidenKamalaHarris2020', 'indices...",0,0,https://twitter.com/twitter/statuses/133592652...,"['joebiden', 'rt', 'agree', 'joebidenkamalahar...",1


Now, we count the number of tweets per label.

In [110]:
Counter(df_clusters.Cluster)

Counter({0: 1464, 1: 12105, 2: 1792, 3: 1283})

We can see that most of the tweets belong to the first cluster. This bias in the data will affect ranking.

### 2. Diversity score

In this section, we assign to each of the rankings by query a diversity score, that indicates how much diversity in the labels there is. First, we retrieve the initial rankings for a list of queries.

In [14]:
queries = ["new coronavirus cases", "covid vaccine", "country with more cases","hospital workers", "global pandemic", "covid deaths", "stayhome in christmas", "test results", "#wearamask"]

In [15]:
rankings = []
for i in range (0,len(queries)):
  #print("Insert your query:\n")
  #query = input()
  ranked_docs = search_tf_idf(queries[i], index, tf, idf)    
  top = 20
  ranking = []

  if(top<=len(ranked_docs)):
    print("\n======================\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
    n=1
    for d_id in ranked_docs[:top] :
      print("{} tweet: {} \n".format(n, d_id))
      print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
      print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
      print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
      print("\n")
      n+=1
      ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
    rankings.append(ranking)
  else:
    print("\n======================\nTop {} results out of {} for the seached query:\n".format(len(ranked_docs), len(ranked_docs)))
    n=1
    for d_id in ranked_docs[:len(ranked_docs)]:
      print("{} tweet: {} \n".format(n, d_id))
      print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
      print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
      print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
      print("\n")
      n+=1
      ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
    rankings.append(ranking)


Top 20 results out of 260 for the seached query:

1 tweet: 14359 

Text: #CORONAVIRUS UPDATE: The state reported 3,817 new cases of the #COVID19 this morning — this is the second-largest daily increase of new cases since the pandemic began. The largest increase was recorded yesterday at 3,880 new cases. https://t.co/ATAq5P7ry1 #Virginia.

Username: 8NEWS. Date: Mon Dec 07 15:53:05 +0000 2020. URL: https://twitter.com/twitter/statuses/1335975628127035394

Retweets: 1. Likes: 3. Hashtags: [{'text': 'CORONAVIRUS', 'indices': [0, 12]}, {'text': 'COVID19', 'indices': [63, 71]}, {'text': 'Virginia', 'indices': [256, 265]}]



2 tweet: 4406 

Text: RT @BangkokPostNews: #ChiangRai reported 6 more #coronavirus #COVID19 cases from Thais returning from Tachilek. The new cases have not been….

Username: alexchoe21. Date: Mon Dec 07 13:54:06 +0000 2020. URL: https://twitter.com/twitter/statuses/1335945683447136263

Retweets: 11. Likes: 0. Hashtags: [{'text': 'ChiangRai', 'indices': [21, 31]}, {'te

Then, we compute the diversity score. This metric has a high value if the ranking contains tweets from a balanced set of clusters. On the other hand, if all tweets in the ranking belong to the same class, then the metric will be low. To do so, we count the number of cluster ocurrences in the ranking and construct a num_clusters-dimensional vector. Also, we calculate the ideal counts for the ranking. For example, if we have 4 clusters and 20 tweets in the ranking, the ideal vector is [5 5 5 5]. To compute the final score, we subtract to 1 the cosine distance between the two vectors: the real count vector and the ideal count vector. In this way, vectors that are close to the ideal ranking will have a diversity score of 1.

In [59]:
def get_ideal_counts(size, num_clusters):
  ideal_count = [size // num_clusters + (1 if x < size % num_clusters else 0)  for x in range(num_clusters)]
  return ideal_count

In [60]:
def get_diversity_score(ranking, num_clusters):
  #divide number of results into cluster buckets
  ideal_count = get_ideal_counts(len(ranking), num_clusters)
  clusters = []
  for element in ranking:
     clusters.append(element[1])
  actual_count = Counter(clusters)
  for i in range(num_clusters):
    if i not in actual_count:
      actual_count[i] = 0

  a = np.array(ideal_count)
  b = np.array(list(actual_count.values()))
  score = 1 - spatial.distance.cosine(a, b)
  #print("The diversity score is " + str(score))
  return score

In [61]:
for ranking in rankings:
  get_diversity_score(ranking, num_clusters)


For example, we can show a toy example of our metric. If the cluster counts are near to the ideal (vector $c$), we get a high diversity score (close to 1).

In [62]:
c = np.array([4, 6, 7, 4])
ideal = np.array([5,5,5,5])
result = 1 - spatial.distance.cosine(c, ideal)
result

0.970725343394151

On the other hand, if the counts are very unbalance, the metric will give a lower result.

In [19]:
d = np.array([19, 1, 0, 0])
ideal = np.array([5,5,5,5])
result = 1 - spatial.distance.cosine(d, ideal)
result

0.5255883312276367

### 3. Re-ranking

In [165]:
threshold = 0.7

In [166]:
def combine_rankings(test_list1, test_list2):
  size_1 = len(test_list1) 
  size_2 = len(test_list2) 
    
  res = [] 
  i, j = 0, 0
    
  while i < size_1 and j < size_2: 
      if test_list1[i] < test_list2[j]: 
        res.append(test_list1[i]) 
        i += 1
    
      else: 
        res.append(test_list2[j]) 
        j += 1
    
  res = res + test_list1[i:] + test_list2[j:] 
  return res

In [228]:
def rerank(ranking, num_clusters, max_ranking, top):
  relevant_i = min(len(ranking), max_ranking)#filter out nonrelevant documents
  print("The maximum rank to consider an element relevant is: " + str(relevant_i))
  ranking = ranking[:] 
  column_names = ["tweet_id", "cluster"]
  df_ranking = pd.DataFrame(data = ranking, columns = column_names)
  cluster_counts = Counter(df_ranking.cluster)
  print(cluster_counts)

  if (top<=len(ranking)):
    print("There are relevant results from %d clusters" %(len(cluster_counts.keys())))
    ranking_new = []
    ideal_count = get_ideal_counts(top, len(cluster_counts.keys()))
    print("ideal count: "+ str(ideal_count))
    i = 0
    for cluster in cluster_counts.keys():
      df_cluster = df_ranking[df_ranking.cluster==cluster]
      if ideal_count[i]<=cluster_counts[cluster]:
        ranking_new = combine_rankings(ranking_new, list(df_cluster.index)[:ideal_count[i]])
      else:
        ranking_new = combine_rankings(ranking_new, list(df_cluster.index)[:cluster_counts[cluster]])
        diff = ideal_count[i] - cluster_counts[cluster]
        ideal_count[(i+1)%num_clusters] +=diff
      i+=1
  else:
    print("There are relevant results from %d clusters" %(len(cluster_counts.keys())))
    ranking_new = []
    ideal_count = get_ideal_counts(len(ranking), len(cluster_counts.keys()))
    i = 0
    for cluster in cluster_counts.keys():
      df_cluster = df_ranking[df_ranking.cluster==cluster]
      if ideal_count[i]<=cluster_counts[cluster]:
        ranking_new = combine_rankings(ranking_new, list(df_cluster.index)[:ideal_count[i]])
      else:
        ranking_new = combine_rankings(ranking_new, list(df_cluster.index)[:cluster_counts[cluster]])
        diff = ideal_count[i] - cluster_counts[cluster]
        ideal_count[(i+1)%num_clusters] +=diff
      i+=1

  df_ranking_new = df_ranking.iloc[ranking_new]
  return list(df_ranking_new.tweet_id)

In [229]:
#a = list(df_ranking[df_ranking.cluster == 0].index)
#b = list(df_ranking[df_ranking.cluster == 1].index)
#combine_rankings(a, b)

In [230]:
df_cluster = df_ranking[df_ranking.cluster==0].reset_index()
df_cluster.head()

,index,tweet_id,cluster
0,5,12613,0
1,6,12098,0
2,8,11404,0
3,12,4396,0
4,13,3945,0


In [231]:
rankings = []
for i in range (0,len(queries)):
  #print("Insert your query:\n")
  #query = input()
  top = 20
  max_ranking = 20000
  diversity_score = 0
  ranking = []
  ranked_docs = search_tf_idf(queries[i], index, tf, idf)
  print("\n======================\n\nLenght of initial ranking: " + str(len(ranked_docs)))

  if(top<=len(ranked_docs)): 
    n=1
    for d_id in ranked_docs[:top] :
      n+=1
      ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
      first_diversity_score = get_diversity_score(ranking[:top], num_clusters)
  else: #if there are less documents thank expected
    n=1
    for d_id in ranked_docs[:len(ranked_docs)]:
      n+=1
      ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
      first_diversity_score = get_diversity_score(ranking[:len(ranked_docs)], num_clusters)


  #check if there is ranking diversity
  if first_diversity_score < threshold: 
    global_ranking = []
    print("The results do not have not enough diversity.")
    for d_id in ranked_docs: #get full ranking
      global_ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])  
        
    new_docs = rerank(global_ranking, num_clusters, max_ranking, top)
    print("\nTop {} results out of {} for the seached query:\n".format(len(new_docs), len(new_docs)))
    n=1
    new_ranking = []
    for d_id in new_docs:
      print("{} tweet: {} \n".format(n, d_id))
      #print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
      #print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
      #print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
      #print("\n")
      new_ranking.append([d_id, df_clusters.Cluster.iloc[d_id]])
      n+=1
    diversity_score = get_diversity_score(new_ranking, num_clusters)
    print("The first diversity score was: " + str(first_diversity_score) + "\n")
    print("New diversity score: " + str(diversity_score))
    print()
  else:
    print("The raking is diverse.")

    if(top<=len(ranked_docs)):
      print("\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
      n=1
      for d_id in ranked_docs[:top] :
        print("{} tweet: {} \n".format(n, d_id))
        #print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
        #print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
        #print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
        #print("\n")
        n+=1
        
    else:
      print("Top {} results out of {} for the seached query:\n".format(len(ranked_docs), len(ranked_docs)))
      n=1
      for d_id in ranked_docs[:len(ranked_docs)]:
        print("{} tweet: {} \n".format(n, d_id))
        #print("Text: {}.\n".format(df_proc.iloc[d_id][0]))
        #print("Username: {}. Date: {}. URL: {}\n".format(df_proc.iloc[d_id][1], df_proc.iloc[d_id][2], df_proc.iloc[d_id][6]))
        #print("Retweets: {}. Likes: {}. Hashtags: {}\n".format(df_proc.iloc[d_id][5], df_proc.iloc[d_id][4], df_proc.iloc[d_id][3]))
        #print("\n")
        n+=1
    print("The diversity score is: " + str(first_diversity_score) + "\n")
    print()

    
  




Lenght of initial ranking: 260
The results do not have not enough diversity.
The maximum rank to consider an element relevant is: 260
Counter({1: 260})
There are relevant results from 1 clusters
ideal count: [20]

Top 20 results out of 20 for the seached query:

1 tweet: 14359 

2 tweet: 4406 

3 tweet: 4983 

4 tweet: 4749 

5 tweet: 4685 

6 tweet: 4679 

7 tweet: 4072 

8 tweet: 3280 

9 tweet: 3089 

10 tweet: 14396 

11 tweet: 977 

12 tweet: 972 

13 tweet: 12062 

14 tweet: 636 

15 tweet: 16521 

16 tweet: 5427 

17 tweet: 192 

18 tweet: 11623 

19 tweet: 1862 

20 tweet: 12659 

The first diversity score was: 0.5

New diversity score: 0.5



Lenght of initial ranking: 75
The results do not have not enough diversity.
The maximum rank to consider an element relevant is: 75
Counter({1: 75})
There are relevant results from 1 clusters
ideal count: [20]

Top 20 results out of 20 for the seached query:

1 tweet: 9556 

2 tweet: 9515 

3 tweet: 9518 

4 tweet: 15983 

5 tweet: 1405